# Creating housing archetypes using K-means method
### Q4 2020-21

## Introduction
This workbook will demonstrate the use of k-means clustering for housing archetype creation. An optimal clustering process is developed on a test dataset though incremental clustering, starting with 1D, then 2D then multi-dimensional clustering. In each increment of clustering different methods are tested and new insight into how these methods perform are found though visualization and internal clustering indices. These findings are then carried on an applied in the next increment of clustering analysis. In the final increment of multi-dimensional clustering the process developed should produce acceptable results: a set of hosing archetypes that are representative of the dataset analyzed. 

### Notes 
#### 
The random state hyper-parameter of k-means is set to a value (42) to produce replicable results during testing; in practice this should be set to 'none'.
#### 
K-value range of 2-11 clusters was predetermined.

### Importing packages 

In [ ]:
#import required packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from numpy import where

from scipy import stats
from kneed import KneeLocator 

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import LocalOutlierFactor

from sklearn.metrics import (silhouette_score, calinski_harabasz_score, davies_bouldin_score)

from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import InterclusterDistance
from yellowbrick.cluster import SilhouetteVisualizer

import plotly.express as px



## Data Source
The data represents a subset of ERS records used for initial analysis. This source is used for each increment of clustering.

In [ ]:
ers_sample_records = pd.read_csv(r"C:\Users\owner\Documents\NRCan\code\practice\InitialHousingData.csv",)
ers_sample_records 

#original amount of obsevations/rows 
og_obs = ers_sample_records.shape[0]

In [ ]:
ers_sample_records.describe()

# 1D Clustering 
Outlier detection methods, normalization methods, and k-means initialization methods were tested and compared with resulting internal indices and scatter plots after cluster analysis.  

## Select Variables
Select a single variable to perform a cluster analysis on.

In [ ]:
# input variable for clustering as cl_variables
cl_variables = ['FloorArea']
# the data is filtered to only include the previously selected variable
test_data = ers_sample_records[cl_variables]


## Data Preparation
Rows (houses) with missing data and unrealistic values are removed. Outliers are detected using three different methods and are subsequently removed.

##Missing values
Dropna from pandas is used to remove rows were at least on element is na. 

In [ ]:
#Remove rows with blank values
test_data_cleaned = test_data.dropna()
#display how many rows removed by comparing to ers_sample_records stats
test_data_cleaned.describe()

###  Inconsistent data removal 
Dependent on the variable. Example: floor area could not be negative.

In [ ]:
filt_data = test_data_cleaned[test_data_cleaned > 0]

### Ensure all values are numerical
K-means can only handle numerical values. 
None of the variables were categorical so this step was not applied.

### Remove outliers

K-means is based off finding the mean of clusters and since means are sensitive to outliers so is k-means. If outliers are not delt with the they can have a large influence in the clustering process that could result in poor partitions that are not representative of the data. Because of this outliers need to be removed properly.

Three different outlier removal techniques are tested: local outlier factor (LOF), z-score, and inter quartile range (IQR). 

The data can be interpreted before and after outlier removal though box plots and statistical measures.

In [ ]:
#boxplot before outlier removal
filt_data.boxplot() 

#### Local outlier factor
Local outlier factor (LOF) values identify an outlier based on the local neighborhood. It gives better results than the global approach to find outliers. A point will be considered as an outlier if it is at a small distance to the extremely dense cluster.

In [ ]:
# Normalize data before lof 
minmax = MinMaxScaler()
scaled_data_lof = minmax.fit_transform(filt_data)

#define the model

lof = LocalOutlierFactor()
lof_pred = lof.fit_predict(scaled_data_lof) 

#extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = filt_data[mask]
print([lof_data])

#print amount of points deleted

outliers_rem = og_obs - lof_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

#plot without after removal

lof_data.boxplot()

#### Z-score 
The z-score method labels an object an outlier depending on its distance from the mean. The distance is measured using standard deviations and is based on the assumption that the data has a gaussian distribution.

In [ ]:
#find absolute value of z-score for each observation
z = np.abs(stats.zscore(filt_data))

#only keep rows in dataframe with all z-scores less than absolute value of 3 
z_data = test_data_cleaned[(z<3).all(axis=1)]

#print amount of outliers removed
z_data

outliers_rem = og_obs - z_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plot after outlier removal
z_data.boxplot()

#### IQR
The IQR can be used to identify outliers by defining limits on the sample values that are a factor k of the IQR below the 25th percentile or above the 75th percentile. The common value for the factor k is the value 1.5 (used below). A factor k of 3 or more can be used to identify values that are extreme outliers or “far outs” when described in the context of box and whisker plots.

In [ ]:
#find Q1, Q3, and interquartile range for each column
Q1 =filt_data.quantile(q=.25)
Q3 = filt_data.quantile(q=.75)
IQR =filt_data.apply(stats.iqr)

#only keep rows in dataframe that have values within 1.5*IQR of Q1 and Q3
iqr_data = filt_data[~((filt_data < (Q1-1.5*IQR)) | (filt_data > (Q3+1.5*IQR))).any(axis=1)]

#print amount of outliers removed

outliers_rem = og_obs - iqr_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plot of IQR outlier removal
iqr_data.boxplot()


## Clustering
The clustering analysis is performed on each set of preprocessed data (3 sets different by outlier detection methods) using a combination of methods for Find the best combination of parameters for clustering. Parameters include outlier removal, scaling, initialization, and optimal amount of clusters.

In [ ]:
# create sets of pre-processed data ready for clustering 
preprocessed_data_sets = [iqr_data,lof_data, z_data]
#list of scalers
standard = StandardScaler()
minimax = MinMaxScaler()
scalers = [standard, minimax]
#list of initializers  
r = 'random'
plus = 'k-means++' 
initalizer = [r, plus]

In [ ]:
# datasets being transformed with standard scaler and random initialization
standard_rand_in=[]
standard_rand_cal=[]
standard_rand_dav=[]
standard_rand_sil=[]

for x in range(len(preprocessed_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(preprocessed_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow method
    
    sse=[] #determine SSE(inertia) for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clamount  
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_rand_in.append(kmeans.inertia_) 
    
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(preprocessed_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(preprocessed_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(preprocessed_data_sets[x], pred_labels)
    
    standard_rand_cal.append(cal_score)
    standard_rand_dav.append(dav_score)
    standard_rand_sil.append(sil)

In [ ]:
#datasets being transformed with minmax scaler and random initialization
mini_rand_in=[]
mini_rand_cal=[]
mini_rand_sil=[]
mini_rand_dav=[]
for x in range(len(preprocessed_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(preprocessed_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
   
    #validation metrics 
    
    mini_rand_in.append(kmeans.inertia_) # can use other validations too 
   
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(preprocessed_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(preprocessed_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(preprocessed_data_sets[x], pred_labels)

    mini_rand_cal.append(cal_score)
    mini_rand_sil.append(sil)
    mini_rand_dav.append(dav_score)

In [ ]:
#datasets being transformed with minmax scaler and k-means++ initialization

mini_plus_in=[]
mini_plus_cal=[]
mini_plus_sil=[]
mini_plus_dav=[]

for x in range(len(preprocessed_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(preprocessed_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow method
    
    sse=[] #determine SSE(inertia) for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined k amount 
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clamount 
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    mini_plus_in.append(kmeans.inertia_) 
    
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(preprocessed_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(preprocessed_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(preprocessed_data_sets[x], pred_labels)
  
    mini_plus_cal.append(cal_score)
    mini_plus_sil.append(sil)
    mini_plus_dav.append(dav_score)
    

  

In [ ]:
#datasets being transformed with standard scaler and k-means++ initialization

standard_plus_in=[]
standard_plus_cal=[]
standard_plus_dav=[]
standard_plus_sil=[]


for x in range(len(preprocessed_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(preprocessed_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clamount
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_plus_in.append(kmeans.inertia_)  
     
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(preprocessed_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(preprocessed_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(preprocessed_data_sets[x], pred_labels)
    
    standard_plus_cal.append(cal_score)
    standard_plus_dav.append(dav_score)
    standard_plus_sil.append(sil)
  

### Internal Index Results 
The resulting internal index scores are summarized in tables below. A larger score for inertia, silhouette, and Calinski-Harabasz indicate better clustering, while a smaller Davies-Bouldin index indicates better clustering.

In [ ]:
#intertia results
results_inertia = pd.DataFrame({ 'st rand':standard_rand_in, 'st plus':standard_plus_in,'min rand':mini_rand_in, 'min plus': mini_plus_in}, index= ['IRQ','LOF','Z-score'])
results_inertia


In [ ]:
#davies bouldin results
results = pd.DataFrame({ 'st rand':standard_rand_dav, 'st plus':standard_plus_dav,'min rand':mini_rand_dav, 'min plus': mini_plus_dav}, index= ['IRQ','LOF','Z-score'])
results

In [ ]:
#silhouette results 
results = pd.DataFrame({ 'st rand':standard_rand_sil, 'st plus':standard_plus_sil,'min rand':mini_rand_sil, 'min plus': mini_plus_sil}, index= ['IRQ','LOF','Z-score'])
results

In [ ]:
#calinski harabasz results
results = pd.DataFrame({ 'st rand':standard_rand_cal, 'st plus':standard_plus_cal,'min rand':mini_rand_cal, 'min plus': mini_plus_cal}, index= ['IRQ','LOF','Z-score'])
results

### Result Visualization 
Based off the resulting internal indices the optimal clustering process included LOF outlier detection, Minmax scaling, inertia (elbow method) for k-value determination, k-means++ initialization. The clusters formed from this method are visualized below.

In [ ]:
#data
data = lof_data

#merging original data with cleaned data for plotting relationships later
lof_data_full = pd.merge(data, ers_sample_records, right_index=True, left_index =True) 
lof_data_full
lof_data_full = lof_data_full.drop(columns=['Rating_x'])

#re-setting index for merging with cluster labels later
data_all = lof_data_full
data_all = data_all.reset_index(drop=True)
data_all


In [ ]:
# scatter plot before clustering to look for intuitive clusters 
sns.pairplot(data_all)

In [ ]:
#scaling with MinMax scaler
minmax = MinMaxScaler()
scaled_features = minmax.fit_transform(data)

#scaled feature into a data frame 
scaled_features = pd.DataFrame(scaled_features)

In [ ]:
#plot histogram of scaled features 
scaled_features.hist(bins=10)

In [ ]:
# before scaling histogram
data.hist(bins=10)

In [ ]:
#clustering

kmeans_kwargs = {
    "init":"k-means++", 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}

sse=[] #determine SSE for 1 to 11 clusters    
for k in range(1,11): 
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
        
#plot elbow method
plt.style.use("fivethirtyeight")
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
        
k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
clamount=k1.elbow
    
#cluster with k determined above 
    
kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)

In [ ]:
# create a dataframe including all variables and cluster labels 
kmeans.fit(scaled_features)
labels = pd.DataFrame(kmeans.labels_,columns=['cluster label'])

cluster1=pd.concat([labels,data_all], axis = 1) 


Evaluate resulting clusters stats. Stats show the distribution of the amount of houses each cluster contains and the variation in the variable used for clustering as well as the other variables the test data included. Optimal clustering can be represented by a smaller std and means/centroids that are dissimilar.

In [ ]:
#cluster 1 dataframe
cluster_0 = cluster1.loc[cluster1['cluster label'] == 0, ['MainWallIns_y']]
cluster_0 = cluster_0.rename(columns={'MainWallIns_y': 'Cluster 0'})

#cluster 1 dataframe
cluster_1 = cluster1.loc[cluster1['cluster label'] == 1, ['MainWallIns_y']]
cluster_1=cluster_1.rename(columns={'MainWallIns_y': 'Cluster 1'})

#cluster 1 dataframe
cluster_2 = cluster1.loc[cluster1['cluster label'] == 2, ['MainWallIns_y']]

#concatinate boxplot for each cluster to compare variation they capture for the certain variable 
#concatinate into a data frame
all_clust = pd.concat([cluster_0,cluster_1, cluster_2], ignore_index=True, axis=1)

#comparison of all cluster stats 
all_clust.describe()

Box plots are a visual representation of the stats. Box plots for each parameter also show how much overlap there is between clusters for each parameter and how compact each cluster is.

In [ ]:
#plot using pandas 
all_clust.boxplot()

Cluster centroid values. These are the mean of each cluster and the values that would represent the synthetic housing archetype.

In [ ]:
#Cluster Centroids 

centroids = minmax.inverse_transform(kmeans.cluster_centers_) # transform scaled cenroids back

centroids

Visualize cluster results through scatter plots silhouette shadow plots and inter-cluster distance plots. 

In [ ]:
#Scatter plot of clusters not scaled
sns.scatterplot('MainWallIns_y', 'YearBuilt', data=cluster1, hue= 'cluster label')

In [ ]:
#scatter plot of scaled values 

#create a dataframe including all variables and cluster labels 
kmeans.fit(scaled_features)
labels = pd.DataFrame(kmeans.labels_,columns=['cluster label'])

data_all_scaled = pd.DataFrame(minmax.fit_transform(data_all), columns = [ 'Air50P', 'Rating', 'YearBuilt', 'FloorArea',
       'MainWallIns_y'])

clusters_scaled = pd.concat([labels, data_all_scaled], axis = 1) 
sns.pairplot(clusters_scaled, hue = 'cluster label')

In [ ]:
# plot all variables (not scaled) against each other to find patterns in clusters  
sns.pairplot(cluster1, hue='cluster label')

In [ ]:
# Initiate the clustering model and visualizer

visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')

visualizer.fit(scaled_features)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure
#In SilhouetteVisualizer plots, clusters with higher scores have wider silhouettes, but clusters 
#that are less cohesive will fall short of the average score across all clusters, which is plotted as a
#vertical dotted red line.

In [ ]:
#inter cluster distance maps
visualizer = InterclusterDistance(kmeans)

visualizer.fit(scaled_features)    # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure
#the closer to centers are in the visualization, the closer they are in the original feature space.

# 2D Clustering 
In this section two variables were clustered using the optimal clustering methods determined in 1D clustering: LOF outlier detection, Minmax scaling, and k-means++ initialization. Methods for optimal amount of clusters were tested and evaluated though 

## Data Preparation
Fist the data is retrieved and prepared. Missing data is removed because k-means does not accept missing values and outliers are also removed because K-means is sensitive to outliers (a mean is easily influenced by extreme value). The preparing will help achieve more robust clustering results.

In [ ]:
#ccreate a variable hold the amount of houses in the test dataset 
og_obs = ers_sample_records.shape[0]

### Variable Selection 
Two variables are selected to perform clustering on. 

In [ ]:
#select variables for 2D clustering 
cl_variables = ['YearBuilt', 'Air50P'] 

# a variable towD_data is created to hold clustering variables specifically for 2D analysis 
twoD_data = ers_sample_records[cl_variables] 
twoD_data

In [ ]:
#review raw data stats 
twoD_data.describe()

### Cleaning Data
Dropna from pandas is used to remove rows were at least on element is na and local outlier factor is applied for outlier detection. 

In [ ]:
#Drop the rows where at least one element is NA.
twoD_data_clean = twoD_data.dropna()
removed = len(twoD_data)-len(twoD_data_clean)

#print amount of rows removed 
print('rows of data dropped:%d' %removed )

In [ ]:
filt_data = twoD_data_clean[twoD_data_clean['YearBuilt'] > 0]
filt_data = filt_data[twoD_data_clean['Air50P'] > 0]
filt_data

### Scaling 
k-means is distance based so for it to consider all attributes as equal and produce unbiased results, they must all have the same scale.

Min-max scaling rescales the data into a given range, in this case 0-1.

In [ ]:
#normalize data before using LOF
minmax = MinMaxScaler()
twoD_data_scaled = minmax.fit_transform(filt_data)

### Outlier removal

Local outlier factor (LOF) values identify an outlier based on the local neighborhood. It gives better results than the global approach to find outliers. A point will be considered as an outlier if it is at a small distance to the extremely dense cluster. 

In [ ]:
#plot variation with box plots before outlier removal to visualize outliers 
filt_data.boxplot()

In [ ]:
#define the model

lof = LocalOutlierFactor()
lof_pred = lof.fit_predict(twoD_data_scaled)  

#extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = filt_data[mask] 

#print amount of points deleated

outliers_rem = og_obs - lof_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

#plot without after removal to visualize outlier removal
lof_data = pd.DataFrame(lof_data, columns = cl_variables)
lof_data.boxplot() 

Also can visualize outlier removal though a scatter plot. The red points are the houses determined outliers that have been removed.

In [ ]:
# create a dataframe of removed outliers for plotting 
lofs_index = where(lof_pred==-1) 
#Filter_df  = twoD_data_clean[twoD_data_clean.index.isin(lofs_index)]
outliers =twoD_data_clean.loc[lofs_index] #datarfame of outliers 
plt.scatter(twoD_data_clean['Air50P'], twoD_data_clean['YearBuilt'])

In [ ]:
#plot removed outliers in red
plt.scatter(twoD_data_clean['Air50P'], twoD_data_clean['YearBuilt'])
plt.scatter(outliers['Air50P'],outliers['YearBuilt'], color='r')
plt.show()

Compare cleaned data stats to raw data to confirm the data is still meaningful. 

In [ ]:
#stats after outlier removal 
lof_data.describe()

In [ ]:
twoD_data.describe()

Normalize with minmax

In [ ]:
minmax = MinMaxScaler()
lof_data = minmax.fit_transform(lof_data)
lof_data = pd.DataFrame(lof_data, columns = cl_variables)

## Clustering 
In this step the prepared data is clustered with the k-means algorithm. K-means++ is used for initialization and the amount of clusters is determined with internal validity measures: Calinski Harabasz, silhouette, inertia, Davies-Bouldin.

In [ ]:
# model used for CH and Silhouette 
model = KMeans(
    init="k-means++",
    n_init=10,
    max_iter=300,
    random_state=42)

#plot Calinski Harabasz 

visualizer = KElbowVisualizer(model, k=(2,10),metric='calinski_harabasz')

visualizer.fit(lof_data)        # Fit the data to the visualizer
visualizer.show()    

#plot silhouette score 
lof
visualizer = KElbowVisualizer(model, k=(2,10),metric='silhouette')

visualizer.fit(lof_data)        # Fit the data to the visualizer
visualizer.show()  

# plot elbow method with inertia/WSS/SSE

sse=[] #determine SSE for 1 to 11 clusters, SSE = WSS
kmeans_kwargs = {
    "init":"k-means++",
    "n_init":10, 
    "max_iter":300,
    "random_state":42,
}

for k in range(1,11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(lof_data)
    sse.append(kmeans.inertia_)
 
plt.style.use("fivethirtyeight")
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

#find the elbow 
k1 = KneeLocator(
    range(1,11),sse,curve="convex", direction="decreasing")
k1.elbow

# plor Db score
DB_score = []

kmeans_kwargs = {
    "init":"k-means++",
    "n_init":10, 
    "max_iter":300,
    "random_state":42,
}

for k in range(2,11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(lof_data)
    pred_labels = kmeans.labels_
    dav_score = davies_bouldin_score(lof_data, pred_labels)
    DB_score.append(dav_score)
   
plt.style.use("fivethirtyeight")
plt.plot(range(2,11), DB_score)
plt.xticks(range(2,11))
plt.xlabel("Number of Clusters")
plt.ylabel("DB")
plt.show()

#find min BD = optimum K
min_index = DB_score.index(min(DB_score))
DB_k =range(2,11)
DB_k = DB_k[min_index]
DB_k 

In [ ]:
k1.elbow

Determining the optimal amount of clusters (k-value) is hard as there is no 'correct' amount, in this case the k-values found from each internal index where used in a cluster analysis(below) then they were visualized using scatter plots and their stats were evaluated. 

After the optimal k value is found the clustering algorithm is ran once more and resulting clusters can be analyzed.

In [ ]:
#final k-means arguments 
kmeans = KMeans(
    init="k-means++",
    n_clusters=3, #change k value here
    n_init=10,
    max_iter=300,
    random_state=42
)

kmeans.fit(lof_data)
labels = pd.DataFrame(kmeans.labels_,columns=['label']) #creating a dataframe to house cluster labels for each house
centroids = kmeans.cluster_centers_ #creating varible to hold centroid values 
centroids_real = minmax.inverse_transform(centroids) #also mean of cluster
kmeans.inertia_

In [ ]:
lof_data_new_index = lof_data.reset_index(drop=True) #new index to allow concatination to line up properly 

clusters = pd.concat([labels,lof_data_new_index], axis = 1)

#using groupby to compare clustering results 
label_group = clusters.groupby(['label']) #make variable for label groups

label_group.get_group(1) #retrieve data frame of group

In [ ]:
#unscaled data 
lof_data_real = minmax.inverse_transform(lof_data)
lof_data_real = pd.DataFrame(lof_data_real, columns = cl_variables )
lof_data_real_new_index = lof_data_real.reset_index(drop=True) #new index to allow concatination to line up properly 

clusters_real = pd.concat([labels,lof_data_real_new_index], axis = 1)

#using groupby to compare clustering results 
label_group_real = clusters_real.groupby(['label']) #make variable for label groups

label_group_real.get_group(1) #retrieve data frame of group

In [ ]:
label_group_real.describe() 

In [ ]:
#using group by to get median and means(centroids) of the resulting clusters
label_group['YearBuilt'].agg(['mean','median'])

In [ ]:
#geting mean and median for just one cluster 
label_group['Air50P'].agg(['mean','median'])

In [ ]:
#stats for all clusters by variable
label_group.describe() 

In [ ]:
#scaling data for plotting
scaled_clusters = clusters.copy() #create a copy so that original not lost or modified 
scaled_clusters[['Air50P', 'YearBuilt']] = minmax.fit_transform(scaled_clusters[['Air50P', 'YearBuilt']]) #put in the cl_varibles or whatever to make it automated
#scaled_clusters

In [ ]:
#plot with centroids
ax=sns.scatterplot(x=scaled_clusters.columns[2],y= scaled_clusters.columns[1], data=scaled_clusters, hue= 'label')
ax=sns.scatterplot(x=centroids[:,1],y=centroids[:,0], s=40, ec='black', legend=False, ax=ax)
plt.show()

In [ ]:
#plot without centroids and  not scaled 
sns.scatterplot(y='YearBuilt',x= 'Air50P', data=clusters, hue= 'label')


# Multidimensional Clustering 
New additions are unrealistic values are removed, option to pre-filter.  

## Data Preparation
Rows with missing data and unrealistic values are removed. Outliers are detected using LOF and are subsequently removed. 

Rename varibales to more intuitive names.

In [ ]:
#rename columns 
ers_sample_records = ers_sample_records.rename(columns={'Air50P':"Airtightness",'FloorArea':"Floor Area", 'MainWallIns':"Main Wall Insulation" , 'YearBuilt':"Year Built", "Rating":"EnerGuide Rating"})

Remove missing houses (rows) that have missing data in at least one column.

In [ ]:
#Drop the rows where at least one element is NA.
cleaned_data = ers_sample_records.dropna()
removed = len(ers_sample_records )-len(cleaned_data)
#print amount of rows removed 
print('rows of data dropped:%d' %removed )

Remove houses (rows) that have inconsistent data. The definition of inconsistent depends on the variable, ensure to change this area when using a new dataset.

In [ ]:
#input bounds for each paramter
filt_data = cleaned_data[cleaned_data['Airtightness'] > 0]
filt_data = filt_data[filt_data['EnerGuide Rating'] > 0]
filt_data = filt_data[filt_data['Year Built'] > 0]
filt_data = filt_data[filt_data['Floor Area'] > 0]
filt_data = filt_data[filt_data['Main Wall Insulation'] > 0]

#print amount of rows removed 
removed = len(cleaned_data )-len(filt_data)
print('rows of data dropped:%d' %removed )

In [ ]:
#inintalize scaler 
scaled = MinMaxScaler()

#scale filtered data in preparation for plotting 
scaled_filt_data = pd.DataFrame(scaled.fit_transform(filt_data))

#plot boxplots before outlier removal

scaled_filt_data.boxplot()

### Outiler Removal 
Local outlier factor algorithm detects the outliers and they are subsequently removed. 

In [ ]:
#define the model

lof = LocalOutlierFactor(n_neighbors=20)
lof_pred = lof.fit_predict(scaled_filt_data) 

#extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = filt_data[mask] 

#print amount of points deleted

outliers_rem = len(filt_data) - lof_data.shape[0]
print ('Outliers removed: %d' %outliers_rem )



To visualize the outlier removal the parameters are scaled using standardscaler(z-score) so that they can be compared on the same axes then plotted with boxplots. 

In [ ]:
#inintalize scaler 
scaled = StandardScaler()

#scale filtered data in preparation for plotting 
scaled_filt_data = pd.DataFrame(scaled.fit_transform(filt_data))

#plot boxplots before outlier removal

scaled_filt_data.boxplot()

In [ ]:
#scale data with outliers removed in preparation for plotting

scaled_parameters = scaled.fit_transform(lof_data)
scaled_parameters = pd.DataFrame(scaled_parameters)

#plot boxplots after outlier removal
scaled_parameters.boxplot()

## Parameter selection 
The minimal amount of parameters that represent the building stock data effectively are selected to be clustered. The parameters that are not selected are removed.

In [ ]:
#insert the column name of the selected parameters that will be held by the varibale parameters
parameters = ['Airtightness', 'Year Built', 'Floor Area',
       'Main Wall Insulation']
# create a new data frame with parameters 
cl_data = lof_data[parameters]

In [ ]:
_1950less = lof_data[(lof_data['Year Built'] <= 1950)]
_1950_1980 = lof_data[(lof_data['Year Built'] >= 1950) & (lof_data['Year Built'] < 1980)]
_1980plus = lof_data[(lof_data['Year Built'] >= 1980)]

In [ ]:
#insert the column name of the selected parameters that will be held by the varibale parameters
#parameters = ['Air50P', 'FloorArea', 'MainWallIns']
# create a new data frame with parameters 
#cl_data = parameters
cl_data

### Parameter scaling 
K-means is distance based so for it to consider all attributes as equal and produce unbiased resutls, they must all have the same scale. Minmax scaling rescales the data into a given range (0-1), this method was chosen from earlier 1D/2D clustering. 

In [ ]:
#initalize scaler
minmax = MinMaxScaler()
#scale parameters
scaled_parameters = minmax.fit_transform(cl_data)


## K-means clustering 

###  Optmial cluster amout determination
Silhouette, Davie-boudin, calinski-harabasz, and inertia are calculated for a range of k-values (2-11) and the k-value. The chosen value is then used as an input for the k-means clustering.

In [ ]:
DB_score = []

kmeans_kwargs = {
    "init":"k-means++",
    "n_init":10, 
    "max_iter":300,
    "random_state":42,
}

for k in range(2,11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_parameters)
    pred_labels = kmeans.labels_
    dav_score = davies_bouldin_score(scaled_parameters, pred_labels)
    DB_score.append(dav_score)
   
plt.style.use("fivethirtyeight")
plt.plot(range(2,11), DB_score)
plt.xticks(range(2,11))
plt.xlabel("Number of Clusters")
plt.ylabel("DB")
plt.show()

#find min BD = optimum K

#amount of clusters for DB
min_index = DB_score.index(min(DB_score))
DB_k =range(2,11)
DB_k = DB_k[min_index]
DB_k 

In [ ]:
model = KMeans(
    init="k-means++",
    n_init=10,
    max_iter=300,
    random_state=42)

visualizer = KElbowVisualizer(model, k=(2,10),metric='silhouette')

visualizer.fit(scaled_parameters)        # Fit the data to the visualizer
visualizer.show()    

#plot elbow method with distortion score

visualizer = KElbowVisualizer(model, k=(1,10))

visualizer.fit(scaled_parameters)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

#plot elbow method Calinski Harabasz 

visualizer = KElbowVisualizer(model, k=(2,10),metric='calinski_harabasz')

visualizer.fit(scaled_parameters)        # Fit the data to the visualizer
visualizer.show()  

### Clustering analysis 
The prepared data is now clustered using the opitmal amount of clusters found in the past step as the n_clusters argument.

In [ ]:
kmeans = KMeans(
    init="k-means++",
    n_clusters=2,#input amount of clusters here
    n_init=10,  
    max_iter=300,
    random_state=42)

kmeans.fit(scaled_parameters)
labels = pd.DataFrame(kmeans.labels_,columns=['Cluster Label']) #creating a dataframe to house cluster labels for each house
centroids = kmeans.cluster_centers_ #creating varible to hold centroid values (scaled) 
centroids_real = minmax.inverse_transform(centroids) #centriods with non-scaled values 

A new dataframe is created adding a column 'label' to show the rows corresponding cluster label decided during k-means clustering. This will allow the resulting partitions to be viewed easier.

In [ ]:
cl_data_new_index = cl_data.reset_index(drop=True) #new index to allow concatenation to line up properly  
#can concatenate along columns but ensure that indices are the same
clusters = pd.concat([labels,cl_data_new_index], axis = 1) 
#make variable for label groups
label_group = clusters.groupby(['Cluster Label']) 

A new dataframe is created adding a column 'label' to show the rows corresponding cluster label decided during k-means clustering. This will allow the resulting partitions to be viewed easier.

In [ ]:
scaled_clusters = pd.DataFrame(scaled_parameters, columns = parameters)
#can concatenate along columns but ensure that indices are the same
scaled_clusters = pd.concat([labels,scaled_clusters], axis = 1) 
#make variable for label groups
scaled_label_group = scaled_clusters.groupby(['Cluster Label']) 

## Results 

### Result Statistics 
The statistics are a way to evaluate the results numerically. 

The amount of objects (houses) in each cluster. 

In [ ]:
label_group.count()

The mean(centroid), standard deviation, and range for each clusters and parameter. Smaller std mean tighter clusters (more compact).

In [ ]:
label_group.agg(['std','min', 'max'])

In [ ]:
scaled_label_group.agg(['std','min', 'max'])

In [ ]:
df_centriods_real = pd.DataFrame(centroids_real, columns = parameters)
df_centriods_real

The centroids values for each parameter. These represent the archetypes characteristics.

### Visualize results 

The resulting clusters plotted using parallel coordinates colour coded by cluster label help to comprehend the distribution of each cluster and how distinct the clusters is.

In [ ]:
fig = px.parallel_coordinates(clusters, color="Cluster Label", 
                             color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

The centriods of the resulting clusters plotted using a parallel coordniates graph.

In [ ]:
#preparing data so that it can be plotted with plotly
mem = label_group.agg(['mean'])
mem.reset_index(level=0, inplace=True)
mem

#create a variable to plot centriods in parallel coordinates graph label

label = ['Cluster Label']
parm = label + parameters
centroids_real_pc =  pd.DataFrame(np.array(mem), columns = parm)


fig = px.parallel_coordinates(centroids_real_pc, color="Cluster Label" )
fig.show()

Box plots for each parameter also show how much overlap there is between clusters for each parameter and how compact each cluster is. IF the IQR of each cluster are not offset they are not distinct. The smaller the IQR and whiskers are the more compact the cluster is. 

In [ ]:
#set up subplot
fig, axes = plt.subplots(1, len(parameters), figsize=(18, 10))

#use a while loop to plot each parameters distribution
i=0
while i != len(parameters):
    
    xx = sns.boxplot(ax=axes[i], x="Cluster Label", y= parameters[i], data=clusters)
    i=i+1

Boxplot of normalized results.

In [ ]:
#set up subplot
fig, axes = plt.subplots(1, len(parameters), figsize=(18, 10))
sns.set_style("whitegrid")
#use a while loop to plot each parameters distribution
i=0
while i != len(parameters):
    
    xx = sns.boxplot(ax=axes[i], x="Cluster Label", y= parameters[i], data=scaled_clusters)
    i=i+1

In [ ]:
visualizer = InterclusterDistance(kmeans)

visualizer.fit(scaled_parameters)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure